In [9]:
import requests
from bs4 import BeautifulSoup

url = 'https://comic.naver.com/webtoon/weekday.nhn'
# 1. 이 url로 HTTP GET요청을 보낸 후, 
#   requests모듈을 사용해서 GET방식으로 요청
# 2. 응답의 text를 html변수에 할당
#   요청의 결과로 반환된 객체를 사용
# 3. soup변수에 BeautifulSoup인스턴스를 생성, 생성 인자로는 위의 html변수 사용
# 4. soup.select('a.title') 한 결과에 웹툰 제목들이 잘 오는지 확인해보기
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.text)

In [12]:
soup.select('a.title')

[<a class="title" href="/webtoon/list.nhn?titleId=183559&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','1')" title="신의 탑">신의 탑</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=648419&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','2')" title="뷰티풀 군바리">뷰티풀 군바리</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=602910&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','3')" title="윈드브레이커">윈드브레이커</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=654774&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','4')" title="소녀의 세계">소녀의 세계</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=597478&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','5')" title="평범한 8반">평범한 8반</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=733766&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','6')" title="인생존망">인생존망</a>,
 <a class="title" href="/webtoon/list.nhn?titleId=694946&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','7')" title="귀전구담">귀전구담<

In [20]:
yumi = 651673
dragon = 568986


def webtoon_detail(title_id):
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    div_comicinfo = soup.select_one('div.comicinfo')
    div_detail = div_comicinfo.select_one('div.detail')
    
    title = div_detail.select_one('h2').contents[0].strip()
    author = div_detail.select_one('span.wrt_nm').get_text(strip=True)
    description = div_detail.select_one('p').get_text('\n', strip=True)
    
    # genre, age
    div_detail_info = div_detail.select_one('p.detail_info')
    genre = div_detail_info.select_one('span.genre').get_text(strip=True)
    age = div_detail_info.select_one('span.age').get_text(strip=True)
    
    # 모든 로직이 이 함수 안에 전부 존재
    return {
        'title': title,
        'description': description,
        'author': author,
        'genre': [g.strip() for g in genre.split(',')],
        'age': age,
    }

In [21]:
webtoon_detail(yumi)

{'title': '유미의 세포들',
 'description': '유미는 지금 무슨 생각을 하고 있을까?\n그녀의 머릿속에서 바쁘게 움직이는 세포들 이야기!',
 'author': '이동건',
 'genre': ['에피소드', '일상', '개그', '로맨스'],
 'age': '12세 이용가'}

In [22]:
webtoon_detail(dragon)

{'title': '용이산다',
 'description': '드래곤이든 인간이든 그저 평범한 판타지 일상물.\n새로운 용(?)물 등장!',
 'author': '초',
 'genre': ['스토리', '일상', '판타지'],
 'age': '전체연령가'}

In [23]:
def episode_list(title_id):
    # title_id에 해당하는 웹툰의 에피소드 목록을 List로 리턴
    # tbody로 찾으면 안나옵니다
    pass

[
    '437화 당신이 받고 있는 시그널 5',
    '436화 당신이 받고 있는 시그널 4',
    '435화 당신이 받고 있는 시그널 3',
    ...
    ...
]

SyntaxError: invalid syntax (<ipython-input-23-5e8b735ff7ef>, line 11)

In [24]:
title_id = 651673
url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html)

In [32]:
for a in soup.select('td.title > a'):
    print(a.get_text())

437화 당신이 받고 있는 시그널 5
436화 당신이 받고 있는 시그널 4
435화 당신이 받고 있는 시그널 3
434화 당신이 받고 있는 시그널 2
433화 당신이 받고 있는 시그널 1
432화 응큼세포의 시그널
431화 루비 퇴근합니다 6
430화 루비 퇴근합니다 5
429화 루비 퇴근합니다 4
428화 루비 퇴근합니다 3


In [50]:
def episode_list(title_id):
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    results = []
    table = soup.select_one('table.viewList')
    tr_list = table.select('tr')
    for tr in tr_list:
        # title클래스를 가진 td
        td_title = tr.select_one('td.title')
        # 가 없으면 넘어간다
        if not td_title:
            continue

        title = td_title.get_text(strip=True)

        td_rating = tr.select_one('td:nth-child(3)')
        rating = td_rating.select_one('strong').get_text(strip=True)

        td_date = tr.select_one('td.num')
        date = td_date.get_text(strip=True)

        episode = {
            'title': title,
            'rating': rating,
            'date': date,
        }
        results.append(episode)
    return results
        
episode_list(dragon)

[{'title': '시즌4 45화', 'rating': '9.54', 'date': '2019.11.22'},
 {'title': '시즌4 44화', 'rating': '9.85', 'date': '2019.11.15'},
 {'title': '시즌4 43화', 'rating': '9.79', 'date': '2019.11.08'},
 {'title': '시즌4 42화', 'rating': '9.85', 'date': '2019.11.01'},
 {'title': '시즌4 41화', 'rating': '9.84', 'date': '2019.10.25'},
 {'title': '시즌4 40화', 'rating': '9.84', 'date': '2019.10.18'},
 {'title': '시즌4 39화', 'rating': '9.80', 'date': '2019.10.11'},
 {'title': '시즌4 38화', 'rating': '9.83', 'date': '2019.10.04'},
 {'title': '시즌4 37화', 'rating': '9.85', 'date': '2019.09.27'},
 {'title': '시즌4 36화', 'rating': '9.80', 'date': '2019.09.20'}]

# 숙제
입력받아 동작하는 웹툰 정보 프로그램 만들기

```
## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 1
> 웹툰명을 입력하세요: 유

# 정보를 가져올 웹툰선택
1. 유일무이 로맨스
2. 윌유메리미
3. 유미의 세포들
4. 당신에게 끌리는 올바른 이유
...
0. 이전
> 웹툰 선택: 3

# 정보 (유미의 세포들)
작가: 이동건
설명: 어쩌고저쩌고....
장르: a, b, c, d

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 2
> 웹툰명을 입력하세요: 유

# 에피소드 목록을 가져올 웹툰선택
1. 유일무이 로맨스
2. 윌유메리미
3. 유미의 세포들
4. 당신에게 끌리는 올바른 이유
...
0. 이전
> 웹툰 선택: 3

# 에피소드 목록 (유미의 세포들)
437화 당신이 받고 있는 시그널 5
 평점: 5.17, 날짜: 2019.12.03
436화 당신이 받고 있는 시그널 4
 평점: 9.58, 날짜: 2019.11.29
...
...
...
...

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 0
```

In [1]:
import re

import requests
from bs4 import BeautifulSoup

### 출력할 텍스트
menu = '''## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기'''
info_msg = '''# 정보를 가져올 웹툰선택'''
episode_list_msg = '''# 에피소드 목록 ({title})'''

### 사용할 함수들
def webtoon_info(title):
    soup = BeautifulSoup(requests.get('https://comic.naver.com/webtoon/weekday.nhn').text)
    a_list = soup.select('a.title[title*="{}"]'.format(title))
    
    results = []
    for a in a_list:
        href = a['href']
        m = re.search(r'titleId=(\d+)', href)
        title_id = m.group(1)
        thumbnail = a.parent.select_one('img')['src']
        title = a.get_text(strip=True)
        cur_info = {
            'title': title,
            'title_id': title_id,
            'link': href,
            'thumbnail': thumbnail,
        }
        results.append(cur_info)
    return results


def webtoon_detail(title_id):
    # title_id에 해당하는 웹툰의 상세정보를 리턴해준다
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    div_comicinfo = soup.select_one('div.comicinfo')
    div_detail = div_comicinfo.select_one('div.detail')
    
    title = div_detail.select_one('h2').contents[0].strip()
    author = div_detail.select_one('span.wrt_nm').get_text(strip=True)
    description = div_detail.select_one('p').get_text('\n', strip=True)
    
    # genre, age
    div_detail_info = div_detail.select_one('p.detail_info')
    genre = div_detail_info.select_one('span.genre').get_text(strip=True)
    age = div_detail_info.select_one('span.age').get_text(strip=True)
    
    # 모든 로직이 이 함수 안에 전부 존재
    return {
        'title': title,
        'description': description,
        'author': author,
        'genre': [g.strip() for g in genre.split(',')],
        'age': age,
    }


def episode_list(title_id):
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    results = []
    table = soup.select_one('table.viewList')
    tr_list = table.select('tr')
    for tr in tr_list:
        # title클래스를 가진 td
        td_title = tr.select_one('td.title')
        # 가 없으면 넘어간다
        if not td_title:
            continue

        title = td_title.get_text(strip=True)

        td_rating = tr.select_one('td:nth-child(3)')
        rating = td_rating.select_one('strong').get_text(strip=True)

        td_date = tr.select_one('td.num')
        date = td_date.get_text(strip=True)

        episode = {
            'title': title,
            'rating': rating,
            'date': date,
        }
        results.append(episode)
    return results


def get_title_id(search_keyword, text='웹툰선택'):
    # 정보를 보고 싶을 때
    search_keyword = input('> 웹툰명을 입력하세요: ').strip()
    results = webtoon_info(search_keyword)

    print('# 검색 결과')
    for index, result in enumerate(results, start=1):
        print(f'{index}: {result["title"]}')
        
    choice = int(input(f'> {text}: '))
    # 선택한 번호 -1번 index의 웹툰이 선택한 웹툰(ex: 2일 경우 1번 index)
    webtoon = results[choice - 1]
    title_id = webtoon['title_id']
    return title_id


while True:
    print(menu)
    val = input('> 입력: ')
    print()
    if val == '1':
        title_id = get_title_id(val)
        info = webtoon_detail(title_id)
        print()
        print(f'# 정보 ({info["title"]})')
        print(f'설명: {info["description"]}')
        print(f'작가: {info["author"]}')
        print(f'장르: {info["genre"]}')
        print(f'연령제한: {info["age"]}')
        print()
            
    elif val == '2':
        # 에피소드 목록을 불러오고 싶을 때
        title_id = get_title_id(val)
        episodes = episode_list(title_id)
        for episode in episodes:
            print(episode['title'])
            print(f' 평점: {episode["rating"]}, 날짜: {episode["date"]}')
        print()
    elif val == '0':
        # 종료
        break

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 2

> 웹툰명을 입력하세요: 유
# 검색 결과
1: 유일무이 로맨스
2: 윌유메리미
3: 유미의 세포들
4: 유미의 세포들
5: 윌유메리미
6: 공유몽
7: 유령극단
> 웹툰선택: 3
438화 당신이 받고 있는 시그널 끝
 평점: 7.83, 날짜: 2019.12.06
437화 당신이 받고 있는 시그널 5
 평점: 5.23, 날짜: 2019.12.03
436화 당신이 받고 있는 시그널 4
 평점: 9.57, 날짜: 2019.11.29
435화 당신이 받고 있는 시그널 3
 평점: 9.79, 날짜: 2019.11.26
434화 당신이 받고 있는 시그널 2
 평점: 9.82, 날짜: 2019.11.22
433화 당신이 받고 있는 시그널 1
 평점: 9.82, 날짜: 2019.11.19
432화 응큼세포의 시그널
 평점: 9.95, 날짜: 2019.11.15
431화 루비 퇴근합니다 6
 평점: 9.98, 날짜: 2019.11.12
430화 루비 퇴근합니다 5
 평점: 9.98, 날짜: 2019.11.08
429화 루비 퇴근합니다 4
 평점: 9.98, 날짜: 2019.11.05

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 1

> 웹툰명을 입력하세요: 유
# 검색 결과
1: 유일무이 로맨스
2: 윌유메리미
3: 유미의 세포들
4: 유미의 세포들
5: 윌유메리미
6: 공유몽
7: 유령극단
> 웹툰선택: 5

# 정보 (윌유메리미)
설명: 외모는 상남자, 마음은 감성소녀 윌.외모는 청순녀, 마음은 터프가이 메리!
서울 부산 띠동갑 커플의 리얼 연애 일기
작가: 마인드C
장르: ['에피소드', '일상', '개그']
연령제한: 전체연령가

## 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기
> 입력: 0



In [20]:
class Webtoon:
    # 이 클래스에서 공통적으로 사용할 변수는 클래스변수로 선언
    URL_WEBTOON_LIST = 'https://comic.naver.com/webtoon/weekday.nhn'
    URL_EPISODE_LIST = 'https://comic.naver.com/webtoon/list.nhn?titleId={id}'
    WEBTOON_LIST_HTML = None
    
    def __init__(self, id, url_thumbnail, title):
        # 객체 초기화 메서드, 초기화시 주어진 매개변수들을 인스턴스의 속성으로 지정
        self.id = id
        self.url_thumbnail = url_thumbnail
        self.title = title
        
        self.author = None
        self.description = None
        self.genres = None
        self.age = None
        
    def __repr__(self):
        # 객체의 표현값
        return f'Webtoon({self.title}, {self.id})'
    
    def get_detail_info(self):
        # 자신의 author, description, genres, age값을 채운다
        pass
    
    def show_info(self):
        # 자신의 id, url_thumbnail, title, author, description, genres, age정보를 출력
        # 출력 예)
        # 유미의 세포들
        #  작가: 누구
        #  설명: 무엇
        #  연령: 12세
        pass
        
    @classmethod
    def search(cls, keyword):
        # keyword가 제목에 포함되는 웹툰 목록을 출력
        # 출력한 목록에서 특정 웹툰을 선택하면, 해당 웹툰의 정보를 가지고 Webtoon인스턴스를 생성하여 반환
        
        # requests.get으로 HTTP요청을 보내는 대신
        # Webtoon클래스에서 1번 요청했으면, 그 결과 (HTML text)를 클래스가 가지고 있기
        
        #  클래스변수는 WEBTOON_LIST_HTML을 사용
        # ex) Webtoon.search('유미')  <- HTTP요청 및 클래스 변수로 HTML텍스트를 저장
        #     Webtoon.search('덴마')  <- 요청하지 않고, 클래스가 가지고 있는 HTML텍스트를 사용
        if not cls.WEBTOON_LIST_HTML:
            print('WEBTOON_LIST_HTML이 비었으므로 HTTP요청!')
            response = requests.get(cls.URL_WEBTOON_LIST)
            cls.WEBTOON_LIST_HTML = response.text
            
        soup = BeautifulSoup(cls.WEBTOON_LIST_HTML)
        css_selector = 'a.title[title*="{}"]'.format(keyword)
        a_list = soup.select(css_selector)

        results = []
        for a in a_list:
            href = a['href']
            m = re.search(r'titleId=(\d+)', href)
            title_id = m.group(1)
            thumbnail = a.parent.select_one('img')['src']
            title = a.get_text(strip=True)
            cur_info = {
                'title': title,
                'title_id': title_id,
                'link': href,
                'thumbnail': thumbnail,
            }
            results.append(cur_info)
        
        print('# 검색결과')
        for index, result in enumerate(results, start=1):
            print(f'{index}: {result["title"]}')
        
        choice = int(input('> 선택: '))
        selected = results[choice - 1]
        # 이 클래스의 생성자를 호출, 그 결과를 리턴
        # instance = Webtoon(id=..., title=..., url_thumbnail=...)
        instance = cls(
            id=selected['title_id'],
            title=selected['title'],
            url_thumbnail=selected['thumbnail'],
        )
        return instance
        
    @property
    def link(self):
        return self.URL_EPISODE_LIST.format(id=self.id)
        
    def get_episode_list(self):
        # 이 웹툰이 가진 에피소드 목록을 리턴해준다
        pass

IndentationError: expected an indented block (<ipython-input-20-c1755da81e13>, line 30)

In [16]:
yumi = Webtoon.search('유')
yumi

WEBTOON_LIST_HTML이 비었으므로 HTTP요청!
# 검색결과
1: 유일무이 로맨스
2: 윌유메리미
3: 유미의 세포들
4: 유미의 세포들
5: 윌유메리미
6: 공유몽
7: 유령극단
> 선택: 3


Webtoon(유미의 세포들, 651673)

In [18]:
denma = Webtoon.search('덴마')
denma

# 검색결과
1: 덴마
2: 덴마
3: 덴마
> 선택: 1


Webtoon(덴마, 119874)

In [ ]:
class Episode:
    def __init__(self):
        pass